In [1]:
# 1) Set the location
%cd ../../code/

/Users/user/Documents/GitHub/paraphrase_py/code


/Users/user/Documents/GitHub/Video-from-Script/youtube_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# 2) import the helpers
# from open_source_paraphrase_df_postprocessing import process_records
from postprocessing import process_records, print_summary
from read_and_write_docs import read_jsonl, write_jsonl
from pathlib import Path

import pandas as pd

In [3]:
# 3) set file paths
input_dir  = "/Volumes/BCross/temp_datasets/author_verification/training/Wiki/Qwen2.5_1.5B_Test_Generation"

In [4]:
def read_all_jsonl(dir_path: str) -> pd.DataFrame:
    """
    Reads all .jsonl files in the given directory, uses read_jsonl() to turn each
    into a DataFrame, and concatenates them into one DataFrame.
    """
    base = Path(dir_path)
    # find all files ending with .jsonl
    jsonl_files = base.glob("*.jsonl")

    dfs = []
    for file in jsonl_files:
        try:
            # your existing function
            df = read_jsonl(str(file))
            dfs.append(df)
        except Exception as e:
            print(f"Warning: failed to read {file}: {e}")

    if dfs:
        # concatenate all dataframes (ignore_index to re-index 0..N-1)
        return pd.concat(dfs, ignore_index=True)
    else:
        # no files found or all failed
        return pd.DataFrame()

In [5]:
# 4) read, process, and write
df = read_all_jsonl(input_dir)

In [6]:
processed_df = process_records(df)

In [7]:
total_rows = len(processed_df)
pct_cleaned = processed_df['text_cleaned'].mean() * 100

cleaned = processed_df[processed_df['text_cleaned'] == 1]
total_cleaned = len(cleaned)

print(f"Out of {total_rows} total rows, {total_cleaned} are cleaned ({pct_cleaned:.2f}%).")

stage_counts = cleaned['clean_stage'].value_counts()

print(f"\nOut of {total_cleaned} cleaned rows, clean_stage breakdown:")
for stage, count in stage_counts.items():
    pct_of_cleaned = (count / total_cleaned) * 100
    print(f"  Stage {stage}: {count} rows ({pct_of_cleaned:.2f}%)")

Out of 1690 total rows, 1690 are cleaned (100.00%).

Out of 1690 cleaned rows, clean_stage breakdown:
  Stage fix_salvage_quotes: 782 rows (46.27%)
  Stage original_ok: 366 rows (21.66%)
  Stage wrap_plain_text: 217 rows (12.84%)
  Stage fix_pythonic: 193 rows (11.42%)
  Stage fix_markdown: 128 rows (7.57%)
  Stage fix_outer_brace: 4 rows (0.24%)


In [8]:
print_summary(processed_df)


Out of 1,690 total rows, 1,690 are cleaned (100.00%).

Out of 1,690 cleaned rows, clean_stage breakdown:
  Stage original_ok: 366 rows (21.66%)
  Stage fix_markdown: 128 rows (7.57%)
  Stage fix_outer_brace: 4 rows (0.24%)
  Stage fix_pythonic: 193 rows (11.42%)
  Stage wrap_plain_text: 217 rows (12.84%)
  Stage fix_salvage_quotes: 782 rows (46.27%)



In [9]:
processed_df.head(2)

,doc_id,orig_doc_id,corpus,author,texttype,text,generated_text,time_sec,tokens_per_sec,clean_text,text_cleaned,clean_stage,parsing_errors
0,142_196_88_228_text_1,known [142.196.88.228 - Text-1].txt,Wiki,142.196.88.228,known,The article that is being referred to via the ...,"{""new_document"": ""A paper linked through the b...",24.778169,46.169674,A paper linked through the broken URL posits t...,1,original_ok,[]
1,142_196_88_228_text_1,known [142.196.88.228 - Text-1].txt,Wiki,142.196.88.228,known,The article that is being referred to via the ...,"```json\n{\n ""new_document"": ""The article lin...",31.469207,39.276490,The article linked through this broken connect...,1,fix_salvage_quotes,[original: Expecting value: line 1 column 1 (c...


In [10]:
cleaned.head(5)

,doc_id,orig_doc_id,corpus,author,texttype,text,generated_text,time_sec,tokens_per_sec,clean_text,text_cleaned,clean_stage,parsing_errors
0,142_196_88_228_text_1,known [142.196.88.228 - Text-1].txt,Wiki,142.196.88.228,known,The article that is being referred to via the ...,"{""new_document"": ""A paper linked through the b...",24.778169,46.169674,A paper linked through the broken URL posits t...,1,original_ok,[]
1,142_196_88_228_text_1,known [142.196.88.228 - Text-1].txt,Wiki,142.196.88.228,known,The article that is being referred to via the ...,"```json\n{\n ""new_document"": ""The article lin...",31.469207,39.276490,The article linked through this broken connect...,1,fix_salvage_quotes,[original: Expecting value: line 1 column 1 (c...
2,142_196_88_228_text_1,known [142.196.88.228 - Text-1].txt,Wiki,142.196.88.228,known,The article that is being referred to via the ...,"```json\n{\n ""new_document"": ""The article men...",26.324054,44.522018,The article mentioned through the faulty link ...,1,fix_markdown,[original: Expecting value: line 1 column 1 (c...
3,142_196_88_228_text_1,known [142.196.88.228 - Text-1].txt,Wiki,142.196.88.228,known,The article that is being referred to via the ...,"{\n ""new_document"": ""A paper linked through a...",35.171929,36.392658,A paper linked through a broken URL posits tha...,1,original_ok,[]
4,142_196_88_228_text_1,known [142.196.88.228 - Text-1].txt,Wiki,142.196.88.228,known,The article that is being referred to via the ...,"```json\n{\n ""new_document"": ""A report linked...",21.483025,51.342863,A report linked through a damaged URL suggests...,1,fix_markdown,[original: Expecting value: line 1 column 1 (c...


In [11]:
cleaned[cleaned['clean_stage']=='check_no_json']

,doc_id,orig_doc_id,corpus,author,texttype,text,generated_text,time_sec,tokens_per_sec,clean_text,text_cleaned,clean_stage,parsing_errors


In [12]:
cleaned[cleaned['clean_stage']=='check_no_json'].loc[52,'text']

KeyError: 52

In [ ]:
cleaned[cleaned['clean_stage']=='check_no_json'].loc[52,'generated_text']

In [ ]:
cleaned.loc[30, 'text']

In [ ]:
cleaned.loc[30, 'generated_text']

In [ ]:
cleaned.loc[30, 'clean_text']

In [ ]:
failed = processed_df[processed_df['text_cleaned'] == 0].reset_index()

In [ ]:
failed

In [ ]:
failed.loc[0, 'text']

In [ ]:
failed.loc[0, 'generated_text']

In [ ]:
failed.loc[0, 'clean_text']

In [ ]:
failed.loc[0, 'parsing_errors']